In [17]:
import pandas as pd
import statsmodels.formula.api as smf

dfcounty = pd.read_csv('US.csv')
df = pd.read_csv('HCS.csv')
df1 = pd.read_csv('HCS-1.csv')
df2 = pd.read_csv('PR.csv')
dfall = pd.concat([df,df1])
dfnew = pd.merge(dfall,dfcounty,on=['City','State'],how='inner')
#print(dfnew)
#dfnew2 = dfnew.drop_duplicates()
dfnew['Crimes']=dfnew['1st quarter']+dfnew['2nd quarter']+dfnew['3rd quarter']+dfnew['4th quarter']
dfcrimes=dfnew[['County','Crimes']]
dfcrimes2=dfcrimes.groupby(['County'])[['Crimes']].sum().reset_index()
dfpres = df2.groupby('County').sum().reset_index()
dfpres2=dfcrimes2.merge(dfpres,on=['County'],how='inner')

# create a fitted model in one line
lm = smf.ols(formula='Crimes ~ Binary', data=dfpres2).fit()

# print the coefficients
lm.summary()





<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Crimes   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     16.85
Date:                Wed, 29 Mar 2017   Prob (F-statistic):           4.29e-05
Time:                        18:50:00   Log-Likelihood:                -12187.
No. Observations:                1394   AIC:                         2.438e+04
Df Residuals:                    1392   BIC:                         2.439e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    528.3889    107.769      4.903      0.000       316.982   739.795
Binary      -477.5603    116.348     -4.105      0.000      -705.796  -249.324
==============================================================================
Omnibus:                     3268.122   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         19233730.543
Skew:                          22.177   Prob(JB):                         0.00
Kurtosis:                     576.736   Cond. No.                         5.13
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
for index, row in df.iterrows():
    print(mydict[str(row['hash'])])